# MASK DESIGN : Vertical Adiabatic Taper

## Device Creation



In [1]:
# Basic classes

import samplemaker.layout as smlay
import samplemaker.makers as sm
from samplemaker.devices import Device, registerDevicesInModule
from samplemaker.plotly_viewers import DeviceInspect, GeomView
from samplemaker.baselib.waveguides import BaseWaveguideSequencer, BaseWaveguidePort
from samplemaker.routers import WaveguideConnect
from samplemaker.shapes import GeomGroup
# Create a simple mask layout
themask = smlay.Mask("Adiabatic Vertical Taper Mask")

Loaded BASELIB_CMARK : Generic cross marker for mask alignment.
Loaded BASELIB_DCPL : Simple symmetric directional coupler
Loaded BASELIB_FGC : Grating coupler demo.
Base library loaded


### First we check the geometry


In [2]:
taper_length = 15
top_length = 30
top_width = 0.500
top_tip_width = 0.05

seq = [
    ["S", top_length/2],
    ["T", taper_length, top_tip_width]
]

sequencer = BaseWaveguideSequencer(seq)
sequencer.reset()
sequencer.options["defaultWidth"] = top_width
sequencer.options["wgLayer"] = 3

top = sm.GeomGroup()
top += sequencer.run()
top += top.copy().mirrorX(0)

GeomView(top)







In [3]:
# Now Bottom 
bottom_length = 15
bottom_width = 1

seq = [
    ["S", top_length/2],
    ["T", taper_length, bottom_width],
    ["S", bottom_length]
]

sequencer = BaseWaveguideSequencer(seq)
sequencer.reset()
sequencer.options["defaultWidth"] = top_width
sequencer.options["wgLayer"] = 1

bot = sm.GeomGroup()
bot += sequencer.run()
bot += bot.copy().mirrorX(0)

GeomView(bot)


geom = top + bot
GeomView(geom)




In [4]:
# taper 2
import numpy as np
taper_length = 15
n_points = 1000
x = np.linspace(0,taper_length, n_points)
m = 2
w1 = 0.5
w2 = 0.05
a = (w1-w2)/(taper_length**m)
w = a*(taper_length-x)**m + w2
geom_test = sm.make_tapered_path(x, np.zeros(n_points), w,3)

GeomView(geom_test, fix_aspect_ratio=False, plot_height = 600)




## Now we realize the device



### Nanobeam

In [161]:

class Nanobeam(Device):
    def initialize(self):
        self.set_name("NANOBEAM")
        self.set_description("Adiabatic Vertical Coupler, InP coupon")

    def parameters(self):
        self.addparameter("taper_length", 15, "Length of each taper")
        self.addparameter("top_length", 30, "Length of top waveguide")
        self.addparameter("top_width", 0.5, "Width of top waveguide")
        self.addparameter("top_tip_width", 0.05, "Width of top waveguide tip")
        self.addparameter("n_points", 1000, "Number of points used in the taper", param_type=int)
        self.addparameter("m", 2, "Top taper power")
        self.addparameter("N_theters", 3, "Number of theters", param_type=int)
        self.addparameter("theter_length", 2, "Length of theters")   
        self.addparameter("theter_width", 0.5, "Width of theters")
        self.addparameter("offset_from_tip", 0.8, "Offset from the tip of the top waveguide as a fraction of the taper length", param_range=[0,1.5])
        self.addparameter("group_id", 0, "Group ID for the device", param_type=int,  param_range=(0, 200))
        self.addparameter("row_id", 0, "Row ID for the device", param_type=int, param_range=(0, 200))
        self.addparameter("col_id", 0, "Column ID for the device", param_type=int, param_range=(0, 200))
    def geom(self):
        p = self.get_params()
        length_total = p["top_length"] + 2*p["taper_length"] 
        
        #top waveguide
        seq = [
            ["S", p["top_length"]/2],
        ]

        sequencer = BaseWaveguideSequencer(seq)
        sequencer.reset()
        sequencer.options["defaultWidth"] = p["top_width"]
        sequencer.options["wgLayer"] = 3

        top = sm.GeomGroup()
        top += sequencer.run()
        
       
        taper_length = p["taper_length"]
        n_points = p["n_points"]
        x = np.linspace(0,p["taper_length"], p["n_points"])
        m = p["m"]
        w1 = p["top_width"]
        w2 = p["top_tip_width"]
        a = (w1-w2)/(taper_length**m)
        w = a*(taper_length-x)**m + w2

        top += sm.make_tapered_path(x, np.zeros(n_points), w,3).translate(p["top_length"]/2, 0)        
        top += top.copy().mirrorX(0)

        #bottom waveguide
        seq = [
            ["S", p["top_length"]/2],
        ]       
       
        XP  = length_total /2
        YP  = 0
        bending_radius = 10

        import math
        def WaveguideConnector(port1, port2, width):
            res = WaveguideConnect(port1, port2, bending_radius)
            if res[0] == True: 
                so = BaseWaveguideSequencer(res[1])
                so.options = sequencer.options
                so.options["defaultWidth"] = width
                g = so.run()
                g.rotate_translate(port1.x0, port1.y0, math.degrees(port1.angle()))
                return g
            else: 
                return GeomGroup()

        p1 = BaseWaveguidePort(-XP, YP, "west",p["top_tip_width"], "p1")
        p2 = BaseWaveguidePort(XP, YP, "east", p["top_tip_width"], "p2")

        p1.connector_function = WaveguideConnector
        p2.connector_function = WaveguideConnector
        self.addlocalport(p1)
        self.addlocalport(p2)
        theters = GeomGroup()
        spacing_between_theters = (p["top_length"] +2*(1-p["offset_from_tip"])*taper_length) / (p["N_theters"] -1)
        
        if p["N_theters"] % 2 == 0:  # even number of theters
            theters += sm.make_rect(spacing_between_theters / 2, 0, p["theter_width"], p["top_width"] / 2 + p["theter_length"], layer=3, numkey=2)
            theters += sm.make_rect(-spacing_between_theters / 2, 0, p["theter_width"], p["top_width"] / 2 + p["theter_length"], layer=3, numkey=2)

            for i in range(1, p["N_theters"] // 2):
                theters += sm.make_rect(spacing_between_theters / 2 + i * spacing_between_theters, 0, p["theter_width"], p["top_width"] / 2 + p["theter_length"], layer=3, numkey=2)
                theters += sm.make_rect(-spacing_between_theters / 2 - i * spacing_between_theters, 0, p["theter_width"], p["top_width"] / 2 + p["theter_length"], layer=3, numkey=2)
        else:
            theters += sm.make_rect(0, 0, p["theter_width"], p["theter_length"]+p["top_width"]/2, layer=3, numkey=2)
            for i in range(1, p["N_theters"] // 2 + 1):
                theters += sm.make_rect(i * spacing_between_theters, 0, p["theter_width"], p["top_width"] / 2 + p["theter_length"], layer=3, numkey=2)
                theters += sm.make_rect(-i * spacing_between_theters, 0, p["theter_width"], p["top_width"] / 2 + p["theter_length"], layer=3, numkey=2)
        theters += theters.copy().mirrorY(0)


        nanobeam = top + theters
        bb = nanobeam.bounding_box()
        rbox=bb.toRect() # rbox contains 1 rectangle size of the bounding box
        rbox.set_layer(4)
        nanobeam = rbox.boolean_difference(nanobeam, 4, 3 )
        str = f"{p['group_id']}_{p['row_id']}_{p['col_id']}"
        text = sm.make_text(0, 2*(p["theter_length"] + p["top_width"]), str, 2, 0.2, to_poly=True, numkey=2, layer=1)
        nanobeam += text
    
        
        #avc += bot  
        return nanobeam
    

coupon = Nanobeam.build()
#DeviceInspect(coupon, fix_aspect_ratio=True)
registerDevicesInModule(__name__)
geom = coupon.geom()
GeomView(geom, fix_aspect_ratio=True, plot_height = 600)


Loaded FULLDEVICE : full device
Loaded NANOBEAM : Adiabatic Vertical Coupler, InP coupon


### Nanobeam with squares


In [183]:
themask = smlay.Mask("Coupons Mask")

class FullDevice(Nanobeam):
    def initialize(self):
        self.set_name("FULLDEVICE")
        self.set_description("full device")
        
    def parameters(self):
        super().parameters()
        self.addparameter("pad_edge", 100, "Edge of the pad")
        self.addparameter("offset_from_corner", 0.1, "Offset from the corner of the pad as a fraction of the pad edge", param_range=[0,0.3])
        self.addparameter("N_theters_pad", 4, "Number of theters in the pad", param_type=int)
        self.addparameter("theter_length_pad", 5, "Length of theters in the pad")
        self.addparameter("theter_width_pad", 5, "Width of theters in the pad")
       

    def geom(self):
        p = self.get_params()
        
        
        nanobeam_dev = Nanobeam.build()
        p2 = nanobeam_dev.get_params()
        for key in p2.keys():
            nanobeam_dev.set_param(key, p[key])
            
        
        nanobeam = nanobeam_dev.geom()  
        
        pad = sm.make_rect(0, 0, p["pad_edge"], p["pad_edge"], layer = 3, numkey=2)
        theters = GeomGroup()

        spacing_between_theters = (p["pad_edge"]*(1-2*p["offset_from_corner"])) / (p["N_theters_pad"] -1)
        if p["N_theters_pad"] % 2 == 0:  # even number of theters
            theters += sm.make_rect(spacing_between_theters / 2, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3, numkey=2)
            theters += sm.make_rect(-spacing_between_theters / 2, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3, numkey=2)

            for i in range(1, p["N_theters_pad"] // 2):
                theters += sm.make_rect(spacing_between_theters / 2 + i * spacing_between_theters, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3, numkey=2)
                theters += sm.make_rect(-spacing_between_theters / 2 - i * spacing_between_theters, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3, numkey=2)
        else:
            theters += sm.make_rect(0, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3)
            for i in range(1, p["N_theters_pad"] // 2 + 1):
                theters += sm.make_rect(i * spacing_between_theters, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3, numkey=2)
                theters += sm.make_rect(-i * spacing_between_theters, p["pad_edge"], p["theter_width_pad"], p["theter_length_pad"], layer=3, numkey=2)
        
        theters += theters.copy().rotate(0, p["pad_edge"]/2, 90)
        theters += theters.copy().rotate(0, p["pad_edge"]/2, 270)

        hole = sm.make_rect(0, 0, 10, 10, layer=2, numkey=1)
        global themask
        themask.addCell("HOLE", hole)
        holes = GeomGroup()

        N_holes_edge = 4
        distance_from_edge = 0.1
        lattice_constant = p["pad_edge"]*(1-2*distance_from_edge)/(N_holes_edge-1)
        square_edge = 10
        dist = 0.1
        N_squares = 4
        spacing = (p["pad_edge"] * (1 - 2 * dist) - N_squares * square_edge) / (N_squares - 1)
        for i in range(N_squares):
            x_pos = -p["pad_edge"] / 2 + p["pad_edge"] * dist + i * (square_edge + spacing)
            for j in range(N_squares):
                y_pos = -p["pad_edge"] / 2 + p["pad_edge"] * dist + j * (square_edge + spacing)
                holes += sm.make_rect(x_pos, y_pos, square_edge, square_edge, layer=2, numkey=1)
        holes.translate(0, p["pad_edge"] / 2)
        pad = pad.boolean_difference(holes, 3, 2)
        
        

        pads = pad + theters
        pads.translate(0, p["theter_length"]+p["top_width"]/2)
        pads.set_layer(3)
        pads+= pads.copy().mirrorY(0)

        bb = pads.bounding_box()
        rbox = bb.toRect()
        rbox.set_layer(4)
        pads = rbox.boolean_difference(pads, 4, 3)

        nanobeam_bb = nanobeam.bounding_box()
        rbox2 = nanobeam_bb.toRect()
        rbox2.set_layer(3)
        pads = pads.boolean_difference(rbox2, 4, 3)

        FullDevice = pads + nanobeam
        Text = FullDevice.select_layer(1)
        Text.translate(0, p["pad_edge"]*1.1)
        
       
        return FullDevice

FullDevice.build()
DeviceInspect(FullDevice, fix_aspect_ratio=True)
registerDevicesInModule(__name__)
geom = FullDevice.build().geom()
GeomView(geom, fix_aspect_ratio=True, plot_height = 600)   

Loaded FULLDEVICE : full device
Loaded NANOBEAM : Adiabatic Vertical Coupler, InP coupon


## Now we realize the Device Table


In [184]:

# Let's import basic stuff
import samplemaker.layout as smlay # used for layout 
import samplemaker.makers as sm # used for drawing
import samplemaker.devices as smdev # used for device function
# Let's use numpy arrays
import numpy as np


# Create a simple mask layout
themask = smlay.Mask("Nanobeam Mask")

geom = sm.GeomGroup()
coupoon = smdev.Device.build_registered("NANOBEAM")

from itertools import product

group_index = 0
element_row_index = 0
def generate_param_dict(**kwargs):
    # Initialize the dictionary that will store lists for each parameter
    param_dict = {key: [] for key in kwargs}
    
    # Get the cartesian product of all parameter values
    param_combinations = product(*kwargs.values())
    
    # Populate the param_dict with the combinations
    for combination in param_combinations:
        for i, key in enumerate(kwargs):
            param_dict[key].append(combination[i])
    
    return param_dict, len(param_dict[key])

# Example usage:
top_length = [10, 15, 20, 30, 40]
theter_width = [0.1, 0.2, 0.3]
theter_length= [1, 2, 3]
N_theters = [2, 3, 4, 5]
taper_length = [18, 19, 20]



# Generate param dictionary with arbitrary parameters
param_cols, N_cols= generate_param_dict(
    theter_width=theter_width,
    theter_length=theter_length,
    N_theters=N_theters
)

param_rows, N_rows = generate_param_dict(
    taper_length=taper_length,
    top_length=top_length,

)

row_id = [i for i in range(N_rows)]
column_id = [i for i in range(N_cols)]

param_rows["row_id"] = row_id
param_cols["col_id"] = column_id



tab = smlay.DeviceTable(coupon,
                        N_rows, N_cols,
                        param_rows,
                        param_cols
                        )


tab.auto_align(20, 20, numkey=5)
tab.set_annotations(smlay.DeviceTableAnnotations("TL=%R0, WL=%R1", "WT=%C0,\n WL=%C1,\nNT=%C2", 
                                                  80, 20, 
                                                  ("top_length",),
                                                  ("theter_width", "theter_length", "N_theters"),
                                                  text_height=1,
                                                  text_width=0.1))

#GeomView(tab.get_geometries().flatten(), fix_aspect_ratio=False)




themask.addDeviceTable(tab, 0, 0)



In [185]:

geom = sm.GeomGroup()
StabilizedNanobeam = smdev.Device.build_registered("FULLDEVICE")

from itertools import product

group_index = 0
element_row_index = 0
def generate_param_dict(**kwargs):
    # Initialize the dictionary that will store lists for each parameter
    param_dict = {key: [] for key in kwargs}
    
    # Get the cartesian product of all parameter values
    param_combinations = product(*kwargs.values())
    
    # Populate the param_dict with the combinations
    for combination in param_combinations:
        for i, key in enumerate(kwargs):
            param_dict[key].append(combination[i])
    
    return param_dict, len(param_dict[key])

# Example usage:
top_length = [10, 15, 20, 30, 40]
theter_width = [0.1, 0.2, 0.3]
theter_length= [1, 2, 3]
N_theters = [2, 3, 4, 5]
taper_length = [18, 19, 20]



# Generate param dictionary with arbitrary parameters
param_cols, N_cols= generate_param_dict(
    theter_width=theter_width,
    theter_length=theter_length,
    N_theters=N_theters
)

param_rows, N_rows = generate_param_dict(
    taper_length=taper_length,
    top_length=top_length,

)

row_id = [i for i in range(N_rows)]
column_id = [i for i in range(N_cols)]

param_rows["row_id"] = row_id
param_cols["col_id"] = column_id



tab = smlay.DeviceTable(StabilizedNanobeam,
                        N_rows, N_cols,
                        param_rows,
                        param_cols
                        )


tab.auto_align(20, 20, numkey=5)
tab.set_annotations(smlay.DeviceTableAnnotations("TL=%R0, WL=%R1", "WT=%C0,\n WL=%C1,\nNT=%C2", 
                                                  80, 20, 
                                                  ("top_length",),
                                                  ("theter_width", "theter_length", "N_theters"),
                                                  text_height=1,
                                                  text_width=0.1))

#GeomView(tab.get_geometries().flatten(), fix_aspect_ratio=False)




themask.addDeviceTable(tab, 5000, 0)



In [186]:

themask.exportGDS()


Opened Nanobeam Mask.gds
Writing structure: NANOBEAM_0001
Writing structure: NANOBEAM_0002
Writing structure: NANOBEAM_0003
Writing structure: NANOBEAM_0004
Writing structure: NANOBEAM_0005
Writing structure: NANOBEAM_0006
Writing structure: NANOBEAM_0007
Writing structure: NANOBEAM_0008
Writing structure: NANOBEAM_0009
Writing structure: NANOBEAM_0010
Writing structure: NANOBEAM_0011
Writing structure: NANOBEAM_0012
Writing structure: NANOBEAM_0013
Writing structure: NANOBEAM_0014
Writing structure: NANOBEAM_0015
Writing structure: NANOBEAM_0016
Writing structure: NANOBEAM_0017
Writing structure: NANOBEAM_0018
Writing structure: NANOBEAM_0019
Writing structure: NANOBEAM_0020
Writing structure: NANOBEAM_0021
Writing structure: NANOBEAM_0022
Writing structure: NANOBEAM_0023
Writing structure: NANOBEAM_0024
Writing structure: NANOBEAM_0025
Writing structure: NANOBEAM_0026
Writing structure: NANOBEAM_0027
Writing structure: NANOBEAM_0028
Writing structure: NANOBEAM_0029
Writing structure:

In [ ]:
from itertools import product

def generate_param_dict(**kwargs):
    # Initialize the dictionary that will store lists for each parameter
    param_dict = {key: [] for key in kwargs}
    
    # Get the cartesian product of all parameter values
    param_combinations = product(*kwargs.values())
    
    # Populate the param_dict with the combinations
    for combination in param_combinations:
        for i, key in enumerate(kwargs):
            param_dict[key].append(combination[i])
    
    return param_dict

# Example usage:
top_lengths = [10, 15, 20]
theter_widths = [0.1, 0.2, 0.3]
theter_lengths = [1, 2, 3]
N_theters = [2, 3, 4, 5]

# Generate param dictionary with arbitrary parameters
param_dict = generate_param_dict(
    top_lengths=top_lengths,
    theter_widths=theter_widths,
    theter_lengths=theter_lengths,
    N_theters=N_theters
)

print(param_dict)
